In [6]:
!pip install python-dotenv

In [ ]:
import openai
import pandas as pd
import re
from dotenv import load_dotenv
import os


class ProductRecommendationChatbot:
    def __init__(self, catalog_path='asus_clean.csv'):
        """
        Initialize chatbot with product catalog
        
        Args:
            catalog_path (str): Path to the CSV file containing product catalog
        """
        # Validate catalog path
        if not os.path.exists(catalog_path):
            raise FileNotFoundError(f"Katalog tidak ditemukan di {catalog_path}")
        
        # Read catalog
        self.catalog = pd.read_csv(catalog_path)
        
        # Validate catalog columns
        required_columns = ['NamaBarang', 'Harga', 'Link', 'terjual', 'DeskripsiBarang']
        missing_columns = [col for col in required_columns if col not in self.catalog.columns]
        if missing_columns:
            raise ValueError(f"Kolom berikut hilang dari katalog: {missing_columns}")
        
        # Preprocess catalog for easier searching
        self.catalog['SearchText'] = self.catalog.apply(
            lambda row: f"{row['NamaBarang']} {row['DeskripsiBarang']}".lower(), 
            axis=1
        )

    def parse_query(self, query):
        """
        Parse query to extract specific search parameters
        """
        query_lower = query.lower()
        
        # Price detection
        price_patterns = [
            r'\b(di bawah|kurang dari|<=)\s*([\d.,]+)\s*(ribu|rb|juta)?',
            r'\b(di atas|lebih dari|>=)\s*([\d.,]+)\s*(ribu|rb|juta)?'
        ]
        
        price = None
        price_comparison = None
        for pattern in price_patterns:
            match = re.search(pattern, query_lower)
            if match:
                price_comparison = match.group(1)
                price_str = match.group(2)
                multiplier = match.group(3) or ''
                
                # Convert price
                price = float(price_str.replace(',', ''))
                if 'ribu' in multiplier or 'rb' in multiplier:
                    price *= 1000
                elif 'juta' in multiplier:
                    price *= 1000000
                break
        
        # Specification detection
        spec_keywords = [
            'core i3', 'core i5', 'core i7', 'core i9', 
            'ryzen 3', 'ryzen 5', 'ryzen 7', 'ryzen 9',
            'rtx', 'gtx', 'gaming', 'ssd', 
            r'ram \d+gb'
        ]
        specs = re.findall('|'.join(spec_keywords), query_lower)
        
        return price, price_comparison, specs

    def find_products(self, query):
        """
        Find products matching query parameters
        """
        price, price_comparison, specs = self.parse_query(query)
        
        # Initial filtering
        filtered_catalog = self.catalog.copy()
        
        # Price filtering
        if price is not None:
            if price_comparison in ['di bawah', 'kurang dari', '<=']:
                filtered_catalog = filtered_catalog[filtered_catalog['Harga'] <= price]
            elif price_comparison in ['di atas', 'lebih dari', '>=']:
                filtered_catalog = filtered_catalog[filtered_catalog['Harga'] >= price]
        
        # Specification filtering
        if specs:
            spec_query = ' '.join(specs)
            mask = filtered_catalog['SearchText'].str.contains('|'.join(specs), case=False)
            filtered_catalog = filtered_catalog[mask]
        
        # Sort by relevance (Terjual as a proxy)
        filtered_catalog = filtered_catalog.sort_values('terjual', ascending=False)
        
        return filtered_catalog.head(5)

    def format_product_recommendations(self, products):
        """
        Format product recommendations for response
        """
        if products.empty:
            return "Maaf, tidak ada produk yang sesuai dengan pencarian Anda."
        
        recommendations = []
        for _, product in products.iterrows():
            recommendation = (
                f"• *{product['NamaBarang']}*\n"
                f"  Harga: Rp{int(product['Harga']):,}\n"
                f"  Terjual: {product['terjual']}\n"
                f"  Deskripsi: {product['DeskripsiBarang']}\n"
                f"  Link: {product['Link']}"
            )
            recommendations.append(recommendation)
        
        return "\n\n".join(recommendations)

    def generate_response(self, query):
        """
        Generate AI-powered response using OpenAI
        """
        # Find matching products
        matched_products = self.find_products(query)
        products_text = self.format_product_recommendations(matched_products)
        
        # Prepare system prompt
        system_prompt = """
        Kamu adalah asisten AI untuk rekomendasi produk laptop di toko kami. 
        Gunakan bahasa santai namun informatif ala . Fokus pada membantu pelanggan 
        menemukan laptop yang sesuai kebutuhan.

        Aturan:
        - Selalu berikan rekomendasi berdasarkan produk yang tersedia
        - Gunakan bahasa ramah dan komunikatif
        - Jika tidak ada produk sesuai, sampaikan dengan sopan
        - Bantu pelanggan dengan pertanyaan spesifik tentang laptop
        - Hindari membahas topik di luar katalog produk
        """
        
        # Generate response using GPT
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": query},
                    {"role": "assistant", "content": f"Produk yang ditemukan:\n{products_text}"},
                    {"role": "user", "content": "Berikan rekomendasi yang sesuai dengan query pelanggan."}
                ],
                max_tokens=300,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        
        except Exception as e:
            return f"Maaf, terjadi kesalahan dalam pemrosesan: {str(e)}"

def main():
    # Load environment variables
    load_dotenv()
    
    # Initialize OpenAI client
    openai.api_key = "YourAPI Here"

    try:
        # Inisialisasi chatbot
        chatbot = ProductRecommendationChatbot()
        
        print("Selamat datang! Saya adalah asisten rekomendasi laptop. Ada yang bisa saya bantu?")
        
        while True:
            user_input = input("\nMasukkan pertanyaan Anda (ketik 'keluar' untuk berhenti): ")
            
            if user_input.lower() in ['keluar', 'exit', 'bye']:
                print("Terima kasih! Semoga harimu menyenangkan.")
                break
            
            # Generate and print response
            response = chatbot.generate_response(user_input)
            print("\nRekomendasi:", response)
    
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

if __name__ == "__main__":
    main()

Selamat datang! Saya adalah asisten rekomendasi laptop. Ada yang bisa saya bantu?



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  laptop di bawah 5 juta



Rekomendasi: Berikut adalah rekomendasi laptop di bawah 5 juta yang dapat menjadi pilihan tepat untuk Anda:

1. **ASUS VivoBook 15 A516KA-FHD451 - Slate Grey**
   - Harga: Rp4,699,000
   - Spesifikasi: Intel Celeron N4500, Intel UHD Graphics, 4GB RAM, 512GB SSD, layar 15.6 inci FHD, Windows 11, Office Home and Student 2021 included.
   - Link: [ASUS VivoBook 15 A516KA-FHD451](https://www.tokopedia.com/asus/asus-vivobook-15-a516ka-fhd451-slate-grey?extParam=src%3Dshop%26whid%3D2383689)

2. **ASUS VivoBook 15 A516KA-HD4522 - Transparent Silver**
   - Harga: Rp4,599,000
   - Spesifikasi: Intel Celeron N4500, Intel UHD Graphics, 4GB RAM, 256GB SSD, layar 15.6 inci HD, Windows 11, Office Home and Student 2021 included.
   - Link: [ASUS VivoBook 15 A516KA-HD4522](https://www.tokopedia.com/asus/asus-vivobook-15-a516ka-hd4522



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  laptop core i3



Rekomendasi: Berdasarkan query pelanggan yang mencari laptop dengan prosesor Core i3, berikut rekomendasi yang sesuai:
1. **ASUS VivoBook A516JAO-VIPS356 - Transparent Silver**
   - Harga: Rp6,699,000
   - Prosesor: Intel Core i3-1005G1
   - RAM: 4GB
   - Storage: 512GB SSD
   - Layar: 15.6 inci FHD
   - Sistem Operasi: Windows 11 Home
   - Link: [Beli di Tokopedia](https://www.tokopedia.com/asus/asus-vivobook-a516jao-vips356-transparent-silver?extParam=src%3Dshop%26whid%3D2383689)

2. **ASUS VivoBook 15 A516JAO-FHD3215 RAM 8GB - Slate Grey**
   - Harga: Rp6,399,000
   - Prosesor: Intel Core i3-1005G1
   - RAM: 8GB
   - Storage: 256GB SSD
   - Layar: 15.6 inci FHD
   - Sistem Operasi: Windows 11 Home
   - Link: [Beli di Tokopedia](https://www.tokopedia.com/asus/asus-vivobook-15-a



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  saya punya banyak uang , butuh laptop gahar untuk gaming , coba rekomendasikan



Rekomendasi: Berdasarkan kebutuhan gaming yang gahar, saya merekomendasikan laptop ASUS ROG Strix G15 G513RW-R77RG6G-O - Eclipse Gray dengan spesifikasi sebagai berikut:
- Prosesor AMD Ryzen 7 6800H yang powerful
- GPU NVIDIA GeForce RTX 3070 Ti untuk performa gaming yang smooth
- Layar 15.6 inci QHD dengan refresh rate 165Hz untuk visual yang jernih
- Memori 16GB dan storage 1TB SSD untuk kecepatan dan kapasitas penyimpanan yang besar
- Keyboard backlit chiclet dengan per-key RGB untuk tampilan yang stylish
- Wi-Fi 6E untuk koneksi yang stabil
- Battery 90WHrs untuk daya tahan yang baik saat bermain game

Laptop ini cocok untuk gaming berat dan akan memberikan pengalaman gaming yang memuaskan. Kamu bisa mendapatkan laptop ini dengan harga Rp29,999,000. Jika tertarik, kamu dapat langsung mengunjungi link berikut: [ASUS ROG Strix G15 G513RW-R77RG6G-O - Eclipse Gray](https://www.tokopedia.com/asus/asus-rog-strix-g15-g513rw-r77rg6g-o-eclipse-gray?extParam=src%3Dshop%26whid%3D238



Masukkan pertanyaan Anda (ketik 'keluar' untuk berhenti):  kakak tau cara buat indomie telor?



Rekomendasi: Untuk membuat Indomie telur, Anda memerlukan laptop yang dapat membantu Anda menemukan resep dan tutorial memasaknya. Dari produk yang tersedia, saya merekomendasikan ASUS VivoBook A516JAO-VIPS356 - Transparent Silver. Laptop ini cocok untuk browsing resep online dan menonton tutorial memasak dengan layar 15.6 inci FHD yang jernih. Dengan prosesor Intel Core i3-1005G1 dan storage 512GB, Anda dapat dengan lancar menjalankan aplikasi dan menyimpan berbagai resep. Selain itu, keyboard chiclet yang nyaman memudahkan Anda untuk mencatat resep. Semoga rekomendasi ini sesuai dengan kebutuhan Anda!
